In [3]:
import sys
import os

# 手动指定模块路径，假设项目位于 Google Drive 的这个位置
module_path = '/content/drive/MyDrive/weather_forecast_project/src'
if module_path not in sys.path:
    sys.path.append(module_path)
print("模块搜索路径已添加：", module_path)


模块搜索路径已添加： /content/drive/MyDrive/weather_forecast_project/src


In [ ]:
# preprocess_and_save_no_dask.py
from data_preprocessor import DataProcessor
import xarray as xr
import numpy as np

# 定义原始数据文件路径
file_path = r"/content/drive/MyDrive/complete_era5_data.nc"

# 直接加载整个数据集到内存
ds = xr.open_dataset(file_path)
ds.load()  # 强制将数据读取到内存中，避免延迟加载

# 实例化 DataProcessor 并处理数据
data_processor = DataProcessor(lead_time=28)
ds_processed = data_processor.load_and_check(ds)

# 创建特征和目标
features_ds = data_processor.create_features(ds_processed)
targets_da = data_processor.create_targets(ds_processed)

# 针对每个变量进行类型转换，以确保为浮点数
converted_features = []
for var in features_ds.data_vars:
    try:
        converted_var = features_ds[var].astype(np.float32)
        converted_features.append(converted_var)
    except Exception as e:
        print(f"变量 {var} 转换为 float32 失败：{e}")
        continue

# 将所有转换后的变量沿新的 'channel' 维度连接，并转置维度顺序
features_combined = xr.concat(converted_features, dim='channel')
features_combined = features_combined.transpose('valid_time', 'latitude', 'longitude', 'channel')

# 针对目标数据进行类型转换
try:
    targets_da = targets_da.astype(np.float32)
except Exception as e:
    print(f"目标数据转换为 float32 失败：{e}")

# 保存为 NetCDF 文件
features_combined.to_netcdf("/content/drive/MyDrive/processed_features.nc")
targets_da.to_netcdf("/content/drive/MyDrive/processed_targets.nc")

print("数据预处理完成，已保存为 NetCDF 格式。")


所有必要的基本变量均存在。
处理缺失值...
变量 't2m' 存在 5742296 个缺失值，已用均值填充。
变量 'msl' 存在 5742296 个缺失值，已用均值填充。
变量 'u10' 存在 5742296 个缺失值，已用均值填充。
变量 'v10' 存在 5742296 个缺失值，已用均值填充。
变量 'tcwv' 存在 5742296 个缺失值，已用均值填充。
变量 'blh' 存在 5742296 个缺失值，已用均值填充。
变量 'sp' 存在 5742296 个缺失值，已用均值填充。
变量 'tp' 存在 5702280 个缺失值，已用均值填充。
创建基础特征...
计算衍生特征: wind_speed
添加时间特征...
创建2米温度距平作为预测目标...
